In [2]:
from tensorflow.keras.models import load_model

# 加載匯出的 .keras
loaded_model = load_model("gru_model1.keras")

# 測試加載的模型是否可以正確運行
test_predictions = loaded_model.predict(X_test)
print("Predictions from loaded model:", test_predictions[:5])

# 將模型輸出進行二值化
binary_predictions = (test_predictions > 0.5).astype(int)
print("Binary Predictions from loaded model:", binary_predictions[:5])


2024-12-28 20:45:47.172297: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2024-12-28 20:45:47.172327: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-12-28 20:45:47.172343: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-12-28 20:45:47.172560: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-28 20:45:47.172595: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


NameError: name 'X_test' is not defined

In [4]:
print(predictions)

[[0.00821969]
 [0.00821969]
 [0.00171407]
 ...
 [0.00222172]
 [0.00220306]
 [0.00223553]]


In [16]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model

# 1. 設定縮放器
scaler = MinMaxScaler(feature_range=(-1, 1))  # 確保縮放範圍與訓練一致

# 2. 讀取新的感測器數據
new_bpm_data = pd.read_csv('test_sensor_bpm (2).csv')  # 替換為你的感測器數據路徑

# 3. 差分處理並去除 NaN
new_bpm_data['Differenced_BPM'] = new_bpm_data['BPM'].diff()
new_bpm_data = new_bpm_data.dropna(subset=['Differenced_BPM']).reset_index(drop=True)

# 4. 初始化縮放器並縮放差分數據
# 必須使用訓練集的 BPM 數據範圍來進行縮放，這裡需要你提供訓練時的數據最小值和最大值
train_min = -1.0  # 替換為訓練數據的最小值
train_max = 1.0   # 替換為訓練數據的最大值
scaler.min_, scaler.scale_ = -1.0, 2 / (train_max - train_min)

new_scaled_data = scaler.transform(new_bpm_data['Differenced_BPM'].values.reshape(-1, 1))

# 5. 創建滑動窗口
observation_window = 300  # 與模型訓練時一致
X_new = []
for i in range(len(new_scaled_data) - observation_window):
    X_new.append(new_scaled_data[i:i + observation_window])

X_new = np.array(X_new)

# 6. 加載模型
model = load_model("gru_model1.keras")

# 7. 批量預測
new_predictions = model.predict(X_new)
binary_predictions = (new_predictions > 0.5).astype(int)

print("Binary Predictions:", binary_predictions.flatten())

# 8. 遞歸預測未來數據
# 修改後的遞歸預測邏輯
def predict_future_values_with_custom_interval(
    model, initial_sequence, observation_window, steps=2, interval=30
):
    """
    使用前 observation_window 數據預測未來 steps 步，每步 interval 秒，判斷是否有異常（1: 有異常，0: 無異常）。
    
    :param model: 預測模型
    :param initial_sequence: 用於預測的初始數據窗口
    :param observation_window: 觀察窗口大小（秒）
    :param steps: 預測未來步數
    :param interval: 每步預測的時間間隔（秒）
    :return: 每步是否有異常的列表
    """
    predictions = []
    current_sequence = initial_sequence.copy()

    for _ in range(steps):
        # 使用當前序列預測未來 interval 秒內的數據
        next_prediction = model.predict(current_sequence, verbose=0)

        # 檢查 interval 秒內是否有異常
        has_anomaly = (next_prediction[:interval] > 0.5).any()  # 檢查 interval 內是否有異常
        predictions.append(1 if has_anomaly else 0)

        # 模擬新數據的到來，更新序列
        next_value = next_prediction[0, 0]  # 使用當前預測值作為序列更新
        current_sequence = np.append(current_sequence[:, 1:, :], [[[next_value]]], axis=1)

    return predictions


# 使用最後的滑動窗口進行預測
initial_sequence = X_new[-1].reshape(1, observation_window, 1)  # 前 5 分鐘的數據
future_predictions = predict_future_values_with_custom_interval(
    model, initial_sequence, observation_window=300, steps=2, interval=30
)

print("Predicted binary values for the next 1 minute (30s intervals):", future_predictions)

88/88 ━━━━━━━━━━━━━━━━━━━━ 6s 66ms/step
Binary Predictions: [0 0 1 ... 1 1 1]
Predicted binary values for the next 1 minute (30s intervals): [1, 1]
